# This notebook works on extracting unique named entities and organizations from KDD papers and passing them lists.  

In [64]:
import pandas as pd
import numpy as np
import nltk
import os
import subprocess
import unicodedata
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from nltk import Tree
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import re
from operator import itemgetter
import polyglot
import string

In [65]:
path        = os.path.abspath(os.getcwd())
TESTDIR     = os.path.normpath(os.path.join(os.path.expanduser("~"),"projects","LC3-Creations", "examples","KDDsample"))


In [149]:
'''
I experienced unicode problems early on.  Everytime I had an error, I scoured the internet for solutions. Here's the credit.



- For Typeerror codes using subprocess to convert pdf2txt output to straight unicode --> http://stackoverflow.com/questions/33283603/python-popen-communicate-str-encodeencoding-utf-8-errors-ignore-cr
- For problems with ASCII characters --> http://stackoverflow.com/questions/175240/how-do-i-convert-a-files-format-from-unicode-to-ascii-using-python
- For unicode characters left in unicode converted to a string  --> http://stackoverflow.com/questions/8689795/how-can-i-remove-non-ascii-characters-but-leave-periods-and-spaces-using-python

'''


a = unicode(subprocess.check_output(['pdf2txt.py',str(os.path.normpath(os.path.join(TESTDIR,"p1005.pdf")))]),errors='ignore')
document = filter(lambda x: x in string.printable,unicodedata.normalize('NFKD', a).encode('ascii','ignore').decode('unicode_escape').encode('ascii','ignore'))

In [150]:
document[:2000]

'Mining Frequent Itemsets through Progressive Sampling\n\nwith Rademacher Averages\n\nMatteo Riondato\n\nDept. of Computer Science\n\nBrown University\n\nProvidence, RI 02912\n\nmatteo@cs.brown.edu\n\nEli Upfal\n\nDept. of Computer Science\n\nBrown University\n\nProvidence, RI 02912\neli@cs.brown.edu\n\nABSTRACT\nWe present an algorithm to extract an high-quality approx-\nimation of the (top-k) Frequent itemsets (FIs) from ran-\ndom samples of a transactional dataset. With high prob-\nability the approximation is a superset of the FIs, and no\nitemset with frequency much lower than the threshold is in-\ncluded in it. The algorithm employs progressive sampling,\nwith a stopping condition based on bounds to the empirical\nRademacher average, a key concept from statistical learning\ntheory. The computation of the bounds uses characteris-\ntic quantities that can be obtained eciently with a sin-\ngle scan of the sample. Therefore, evaluating the stopping\ncondition is fast, and does not re

In [68]:
"returns named entity chunks in a given text"
tagged = nltk.pos_tag(nltk.word_tokenize(re.sub('[\s]'," ", document)))
entities = nltk.chunk.ne_chunk(tagged)
# Another entity extractor
st = StanfordNERTagger('/Users/linwood/stanford-corenlp-full-2015-04-20/classifiers/english.muc.7class.distsim.crf.ser.gz',
       '/Users/linwood/stanford-corenlp-full-2015-04-20/stanford-corenlp-3.5.2.jar',
       encoding='utf-8')
tokenized_text = word_tokenize(re.sub('[\s]'," ", document))
stanentities = st.tag(tokenized_text)

Below, I established two lists to hold the values that I extract from the text.  This itemgetter function will check for unique values.  First, I iterate over the extracted entities and see if the objects is a nltk.tree.Tree with a "Person" label.  If it is, and the length is equal to 1 (first or last name only), I append that value to the list. If it's larger, I iterate of the entity tree and pull out the first value only using itemgetter.  Then, I join the values from the list and append it to the destination list.  

# Getting a list out of NLTK's standard NE chunker

In [69]:
from operator import itemgetter

persons = []
organizations = []
locations =[]

for l in entities:
    if isinstance(l,nltk.tree.Tree):
        if l.label() == 'PERSON':
            if len(l)== 1:
                if l[0][0] in persons:
                    pass
                else:
                    persons.append(l[0][0])
            else:
                if " ".join(map(itemgetter(0), l)) in persons:
                    pass
                else:
                    persons.append(" ".join(map(itemgetter(0), l)))
                    
for o in entities:
    if isinstance(o,nltk.tree.Tree):
        if o.label() == 'ORGANIZATION':
            if len(o)== 1:
                if o[0][0] in organizations:
                    pass
                else:
                    organizations.append(o[0][0])
            else:
                if " ".join(map(itemgetter(0), o)) in organizations:
                    pass
                else:
                    organizations.append(" ".join(map(itemgetter(0), o)))
                    
for o in entities:
    if isinstance(o,nltk.tree.Tree):
        if o.label() == 'LOCATION':
            if len(o)== 1:
                if o[0][0] in locations:
                    pass
                else:
                    locations.append(o[0][0])
            else:
                if " ".join(map(itemgetter(0), o)) in locations:
                    pass
                else:
                    locations.append(" ".join(map(itemgetter(0), o)))
                    
                
print persons
print
print
print organizations
print
print
print locations

['Local Intrinsic Dimensionality Laurent Amsaleg Equipe LINKMEDIA', 'Rennes', 'Campus Universitaire', 'Rennes Cedex', 'Stphane Girard Equipe MISTIS', 'Furon Equipe LINKMEDIA', 'Oussama Chelly National Institute', 'Michael E. Houle National Institute', 'Michael Nett Google', 'Houle', 'Karger', 'Subject Descriptors', 'Parameter', 'Permissions', 'Keywords', 'Isometric Mapping', 'Linear Embedding', 'Principal', 'Analysis', 'Weibull', 'Maximum', 'Hill', 'Pr', 'Fisher', 'Tippett', 'Gnedenko', 'Given', 'Ruhls', 'J', 'M', 'Hein', 'P', 'D', 'Uniformly', 'Amsterdam Library', 'Object Images', 'Likewise', 'Faster', 'D IDMLE', 'Hein Takens', 'Dataset', 'Data', 'Secular', 'Residual Life Time', 'Cambridge University Press', 'J. Fauqueur', 'Pattern Recogn', 'Statistical Modeling', 'Extreme Values', 'Hero', 'Sys', 'Smallest Member', 'Math', 'Cambridge Phil', 'Fraga Alves', 'Portugalia Mathematica', 'Terme Maximum', 'Serie Aleatoire', 'Ann', 'Gupta', 'Audibert', 'Inlierness', 'Technical Report', 'Nett',

In [70]:
from operator import itemgetter

for o in entities:
    if isinstance(o,nltk.tree.Tree):
        if o.label() == 'ORGANIZATION' or o.label() == 'GPE':
            if len(o)>1:
                print " ".join(map(itemgetter(0), o))

INRIA Grenoble
France Inovalle
Component Analysis
Correlation Dimension
Likelihood Estimation
ID2 X
Distance Distributions
Real Data
Swiss Roll
IDANN_SIFT1B35 Dataset
Real Data
JSPS Kakenhi Kiban
Generic Image Retrieval
Asilomar Conf
Frequency Distribution
Methods of Stat.
Bounded Geometries
Distance Distributions
Document Recognition
Kernel Eigenvalue Problem
ACM Trans
Intrinsic Dimensional Outlier


I tried to iterate over the extracted list of entities to get a better break between person's and their university name.  

In [71]:
tokens = [nltk.word_tokenize(l) for l in persons]
fin = [nltk.chunk.ne_chunk(nltk.pos_tag(l)) for l in tokens]
fin;

In [72]:
new =[word_tokenize(l) for l in persons]
stan = [st.tag(l) for l in new]
stan;

# Creating lists of named entities from Stanford's NER model

This function looks though an extracted stanford ner list, and finds continuous entitiy labels.  This should create first name, last name records of entities.  

In [73]:
def get_continuous_chunks(tagged_sent):
    continuous_chunk = []
    current_chunk = []

    for token, tag in tagged_sent:
        if tag != "O":
            current_chunk.append((token, tag))
        else:
            if current_chunk: # if the current chunk is not empty
                continuous_chunk.append(current_chunk)
                current_chunk = []
    # Flush the final current_chunk into the continuous_chunk, if any.
    if current_chunk:
        continuous_chunk.append(current_chunk)
    return continuous_chunk

ne_tagged_sent = [('Rami', 'PERSON'), ('Eid', 'PERSON'), ('is', 'O'), ('studying', 'O'), ('at', 'O'), ('Stony', 'ORGANIZATION'), ('Brook', 'ORGANIZATION'), ('University', 'ORGANIZATION'), ('in', 'O'), ('NY', 'LOCATION')]

named_entities = get_continuous_chunks(stanentities)
named_entities_str = [" ".join([token for token, tag in ne]) for ne in named_entities]
named_entities_str_tag = [(" ".join([token for token, tag in ne]), ne[0][1]) for ne in named_entities]


In [74]:
named_entities = get_continuous_chunks(stanentities)
named_entities_str = [" ".join([token for token, tag in ne]) for ne in named_entities]
named_entities_str_tag

[(u'Estimating Local Intrinsic Dimensionality Laurent Amsaleg Equipe LINKMEDIA',
  u'ORGANIZATION'),
 (u'CNRSIRISA Rennes', u'LOCATION'),
 (u'France Campus Universitaire de Beaulieu 35042 Rennes Cedex',
  u'ORGANIZATION'),
 (u'France', u'LOCATION'),
 (u'Stphane Girard Equipe MISTIS', u'PERSON'),
 (u'INRIA Grenoble', u'LOCATION'),
 (u'France Inovalle', u'LOCATION'),
 (u'France Teddy Furon Equipe LINKMEDIA', u'ORGANIZATION'),
 (u'INRIAIRISA Rennes', u'LOCATION'),
 (u'France Campus Universitaire de Beaulieu 35042 Rennes Cedex',
  u'ORGANIZATION'),
 (u'France', u'LOCATION'),
 (u'Kawarabayashi National Institute of Informatics', u'ORGANIZATION'),
 (u'Japan', u'LOCATION'),
 (u'Hitotsubashi', u'LOCATION'),
 (u'Chiyoda-ku Tokyo', u'LOCATION'),
 (u'Japan', u'LOCATION'),
 (u'Oussama Chelly National Institute of Informatics', u'ORGANIZATION'),
 (u'Japan', u'LOCATION'),
 (u'Hitotsubashi', u'LOCATION'),
 (u'Chiyoda-ku Tokyo', u'LOCATION'),
 (u'Japan', u'LOCATION'),
 (u'Michael E. Houle National Ins

In [75]:
compare=[]
for l,m in named_entities_str_tag:
    l=re.sub('(\])'," ",l)
    if m == 'PERSON':
        if l in compare:
            pass
        else:
            compare.append(l)
    else:
        pass
compare

[u'Stphane Girard Equipe MISTIS',
 u'Michael Nett Google',
 u'Houle',
 u'Karger',
 u'Weibull',
 u'Fisher',
 u'Tippett',
 u'Balkema',
 u'Haan',
 u'Iverson',
 u'Hein',
 u'Mobius',
 u'ANN SIFT1B',
 u'Hein Takens',
 u'L. Amsaleg',
 u'M. E. Houle',
 u'K. Kawarabayashi',
 u'A. Balkema',
 u'L. de Haan',
 u'  N. Bingham',
 u'  N. Boujemaa',
 u'J. Fauqueur',
 u'M. Ferecatu',
 u'F. Fleuret',
 u'V. Gouet',
 u'B. LeSaux',
 u'H. Sahbi',
 u'  C. Bouveyron',
 u'G. Celeux',
 u'S. Girard',
 u'  J. Bruske',
 u'G. Sommer',
 u'A. Vinciarelli',
 u'  S. Coles',
 u'S. Chawla',
 u'  R. A. Fisher',
 u'L. H. C. Tippett',
 u'  M. I. Fraga Alves',
 u'T. Lin',
 u'M. I. Gomes',
 u'B. V. Gnedenko',
 u'A. Gupta',
 u'R. Krauthgamer',
 u'J. R. Lee',
 u'  M. Hein',
 u'B. M. Hill',
 u'Hubness',
 u'H. Kashima',
 u'M. Nett',
 u'X. Ma',
 u'V. Oria',
 u'  H. Jegou',
 u'R. Tavenard',
 u'M. Douze',
 u'I. Jollie',
 u'  D. R. Karger',
 u'M. Ruhl',
 u'J. Karhunen',
 u'J. Joutsensalo',
 u'  Y. LeCun',
 u'L. Bottou',
 u'Y. Bengio',

In [80]:
len(set(compare) & set(p29pdf_allauthors))

57

In [81]:
len(p29pdf_allauthors)

113

In [ ]:
list1 = range(10)

In [ ]:
list2 = [i for i in xrange(7,17,1)]


In [ ]:
set(list1) & set(list2)

In [ ]:
def parts_of_speech(corpus):
    "returns named entity chunks in a given text"
    tagged = nltk.pos_tag(nltk.word_tokenize(corpus))
    entities = nltk.chunk.ne_chunk(tagged)
    # Another entity extractor
    st = StanfordNERTagger('/Users/linwood/stanford-corenlp-full-2015-04-20/classifiers/english.muc.7class.distsim.crf.ser.gz',
           '/Users/linwood/stanford-corenlp-full-2015-04-20/stanford-corenlp-3.5.2.jar',
           encoding='utf-8')
    tokenized_text = word_tokenize(corpus)
    stanentities = st.tag(tokenized_text)
    return entities
def find_entities(chunks):
    "given list of tagged parts of speech, returns unique named entities"

    def traverse(tree):
        "recursively traverses an nltk.tree.Tree to find named entities"
        entity_names = []
    
        if hasattr(tree, 'node') and tree.node:
            if tree.node == 'NE':
                entity_names.append(' '.join([child[0] for child in tree]))
            else:
                for child in tree:
                    entity_names.extend(traverse(child))
    
        return entity_names
    
    named_entities = []
    
    for chunk in chunks:
        entities = sorted(list(set([word for tree in chunk
                            for word in traverse(tree)])))
        for e in entities:
            if e not in named_entities:
                named_entities.append(e)
    return named_entities

In [ ]:
import os
os.getcwd()

# Extracting entities and creating lists using Polyglot

In [162]:
from polyglot.text import Text
e=Text(re.sub('[\s]'," ",document[:10000])).entities

This code iterates over the polyglot extracted entities and creates a list of person, locations, and organizations

In [ ]:
import itertools
import unicodedata

def extraction(corpus):
    
    # extract entities from a single string; remove whitespace characters
    try:
        e = Text(re.sub('[\s]'," ",corpus)).entities
    except:
        pass #e = Text(re.sub("(r'(x0)'," ","(re.sub('[\s]'," ",corpus)))).entities
    
    current_person =[]
    persons =[]
    current_org=[]
    organizations=[]
    current_loc=[]
    locations=[]

    for l in e:
        if l.tag == 'I-PER':
            for m in l:
                current_person.append(unicodedata.normalize('NFKD', m).encode('ascii','ignore'))
            else:
                    if current_person: # if the current chunk is not empty
                        persons.append(" ".join(current_person))
                        current_person = []
        elif l.tag == 'I-ORG':
            for m in l:
                current_org.append(unicodedata.normalize('NFKD', m).encode('ascii','ignore'))
            else:
                    if current_org: # if the current chunk is not empty
                        organizations.append(" ".join(current_org))
                        current_org = []
        elif l.tag == 'I-LOC':
            for m in l:
                current_loc.append(unicodedata.normalize('NFKD', m).encode('ascii','ignore'))
            else:
                    if current_loc: # if the current chunk is not empty
                        locations.append(" ".join(current_loc))
                        current_loc = []
    results = {}
    results['persons']=persons
    results['organizations']=organizations
    results['locations']=locations
    
    return results

In [ ]:
extraction(document)['persons']

In [ ]:
document;

In [ ]:
regexp = re.compile("REFERENCES(.*)$")
references = Text(regexp.search(re.sub('[\s]'," ",document)).group(1)).entities

In [ ]:
regexp1 = re.compile("REFERENCES(.*)$")
references = Text(regexp.search(re.sub('[\s]'," ",document)).group(1)).entities

In [ ]:
len(extraction(regexp.search(re.sub('[\s]'," ",document)).group(1))['persons'])

# Truth Sets to test extraction accuracy

In [78]:
#p19.pdf

p19pdf_authors=['Tim Althoff','Xin Luna Dong','Kevin Murphy','Safa Alai','Van Dang','Wei Zhang']
p19pdf_author_organizations=['Computer Science Department','Stanford University','Google']
p19pdf_author_locations=['Stanford, CA','Stanford','CA','Google','1600 Amphitheatre Parkway, Mountain View, CA 94043','1600 Amphitheatre Parkway','Mountain View']

p19pdf_references_authors =['A. Ahmed', 'C. H. Teo', 'S. Vishwanathan','A. Smola','J. Allan', 'R. Gupta', 'V. Khandelwal',
                           'D. Graus', 'M.-H. Peetz', 'D. Odijk', 'O. de Rooij', 'M. de Rijke','T. Huet', 'J. Biega', 
                            'F. M. Suchanek','H. Ji', 'T. Cassidy', 'Q. Li','S. Tamang', 'A. Kannan', 'S. Baker', 'K. Ramnath', 
                            'J. Fiss', 'D. Lin', 'L. Vanderwende',  'R. Ansary', 'A. Kapoor', 'Q. Ke', 'M. Uyttendaele',
                           'S. M. Katz','A. Krause','D. Golovin','J. Leskovec', 'A. Krause', 'C. Guestrin', 'C. Faloutsos', 
                            'J. VanBriesen','N. Glance','J. Li','C. Cardie','J. Li','C. Cardie','C.-Y. Lin','H. Lin','J. A. Bilmes'
                           'X. Ling','D. S. Weld', 'A. Mazeika', 'T. Tylenda','G. Weikum','M. Minoux', 'G. L. Nemhauser', 'L. A. Wolsey',
                            'M. L. Fisher','R. Qian','D. Shahaf', 'C. Guestrin','E. Horvitz','T. Althoff', 'X. L. Dong', 'K. Murphy', 'S. Alai',
                            'V. Dang','W. Zhang','R. A. Baeza-Yates', 'B. Ribeiro-Neto', 'D. Shahaf', 'J. Yang', 'C. Suen', 'J. Jacobs', 'H. Wang', 'J. Leskovec',
                           'W. Shen', 'J. Wang', 'J. Han','D. Bamman', 'N. Smith','K. Bollacker', 'C. Evans', 'P. Paritosh', 'T. Sturge', 'J. Taylor',
                           'R. Sipos', 'A. Swaminathan', 'P. Shivaswamy', 'T. Joachims','K. Sprck Jones','G. Calinescu', 'C. Chekuri', 'M. Pl','J. Vondrk',
                           'F. M. Suchanek', 'G. Kasneci','G. Weikum', 'J. Carbonell' ,'J. Goldstein','B. Carterette', 'P. N. Bennett', 'D. M. Chickering',
                            'S. T. Dumais','A. Dasgupta', 'R. Kumar','S. Ravi','Q. X. Do', 'W. Lu', 'D. Roth','X. Dong', 'E. Gabrilovich', 'G. Heitz', 'W. Horn', 
                            'N. Lao', 'K. Murphy',  'T. Strohmann', 'S. Sun','W. Zhang', 'M. Dubinko', 'R. Kumar', 'J. Magnani', 'J. Novak', 'P. Raghavan','A. Tomkins',
                           'U. Feige','F. M. Suchanek','N. Preda','R. Swan','J. Allan', 'T. Tran', 'A. Ceroni', 'M. Georgescu', 'K. D. Naini', 'M. Fisichella',
                           'T. A. Tuan', 'S. Elbassuoni', 'N. Preda','G. Weikum','Y. Wang', 'M. Zhu', 'L. Qu', 'M. Spaniol', 'G. Weikum',
                           'G. Weikum', 'N. Ntarmos', 'M. Spaniol', 'P. Triantallou', 'A. A. Benczr',  'S. Kirkpatrick', 'P. Rigaux','M. Williamson',
                           'X. W. Zhao', 'Y. Guo', 'R. Yan', 'Y. He','X. Li']
p19pdf_allauthors=['Tim Althoff','Xin Luna Dong','Kevin Murphy','Safa Alai','Van Dang','Wei Zhang','A. Ahmed', 'C. H. Teo', 'S. Vishwanathan','A. Smola','J. Allan', 'R. Gupta', 'V. Khandelwal',
                           'D. Graus', 'M.-H. Peetz', 'D. Odijk', 'O. de Rooij', 'M. de Rijke','T. Huet', 'J. Biega', 
                            'F. M. Suchanek','H. Ji', 'T. Cassidy', 'Q. Li','S. Tamang', 'A. Kannan', 'S. Baker', 'K. Ramnath', 
                            'J. Fiss', 'D. Lin', 'L. Vanderwende',  'R. Ansary', 'A. Kapoor', 'Q. Ke', 'M. Uyttendaele',
                           'S. M. Katz','A. Krause','D. Golovin','J. Leskovec', 'A. Krause', 'C. Guestrin', 'C. Faloutsos', 
                            'J. VanBriesen','N. Glance','J. Li','C. Cardie','J. Li','C. Cardie','C.-Y. Lin','H. Lin','J. A. Bilmes'
                           'X. Ling','D. S. Weld', 'A. Mazeika', 'T. Tylenda','G. Weikum','M. Minoux', 'G. L. Nemhauser', 'L. A. Wolsey',
                            'M. L. Fisher','R. Qian','D. Shahaf', 'C. Guestrin','E. Horvitz','T. Althoff', 'X. L. Dong', 'K. Murphy', 'S. Alai',
                            'V. Dang','W. Zhang','R. A. Baeza-Yates', 'B. Ribeiro-Neto', 'D. Shahaf', 'J. Yang', 'C. Suen', 'J. Jacobs', 'H. Wang', 'J. Leskovec',
                           'W. Shen', 'J. Wang', 'J. Han','D. Bamman', 'N. Smith','K. Bollacker', 'C. Evans', 'P. Paritosh', 'T. Sturge', 'J. Taylor',
                           'R. Sipos', 'A. Swaminathan', 'P. Shivaswamy', 'T. Joachims','K. Sprck Jones','G. Calinescu', 'C. Chekuri', 'M. Pl','J. Vondrk',
                           'F. M. Suchanek', 'G. Kasneci','G. Weikum', 'J. Carbonell' ,'J. Goldstein','B. Carterette', 'P. N. Bennett', 'D. M. Chickering',
                            'S. T. Dumais','A. Dasgupta', 'R. Kumar','S. Ravi','Q. X. Do', 'W. Lu', 'D. Roth','X. Dong', 'E. Gabrilovich', 'G. Heitz', 'W. Horn', 
                            'N. Lao', 'K. Murphy',  'T. Strohmann', 'S. Sun','W. Zhang', 'M. Dubinko', 'R. Kumar', 'J. Magnani', 'J. Novak', 'P. Raghavan','A. Tomkins',
                           'U. Feige','F. M. Suchanek','N. Preda','R. Swan','J. Allan', 'T. Tran', 'A. Ceroni', 'M. Georgescu', 'K. D. Naini', 'M. Fisichella',
                           'T. A. Tuan', 'S. Elbassuoni', 'N. Preda','G. Weikum','Y. Wang', 'M. Zhu', 'L. Qu', 'M. Spaniol', 'G. Weikum',
                           'G. Weikum', 'N. Ntarmos', 'M. Spaniol', 'P. Triantallou', 'A. A. Benczr',  'S. Kirkpatrick', 'P. Rigaux','M. Williamson',
                           'X. W. Zhao', 'Y. Guo', 'R. Yan', 'Y. He','X. Li']

print "There are %r authors" % len(p19pdf_authors)
print  # white space
print "There are %r author organizations" %len(p19pdf_author_organizations)
print 
print "There are %r author locations" % len(p19pdf_author_locations)
print  
print "There are %r authors in the references" %len(p19pdf_references_authors)

There are 6 authors

There are 3 author organizations

There are 7 author locations

There are 152 authors in the references


In [77]:
#p29.pdf

p29pdf_authors=['Laurent Amsaleg','Stéphane Girard','Oussama Chelly','Teddy Furon','Michael E. Houle','Ken-ichi Kawarabayashi',
               'Michael Nett']
p29pdf_author_organizations=['Equipe LINKMEDIA','Campus Universitaire de Beaulieu','CNRS/IRISA Rennes','National Institute of Informatics',
                             'Equipe MISTIS INRIA','Google']
p29pdf_author_locations=['Campus Universitaire de Beaulieu','35042 Rennes Cedex, France','France','-1-2 Hitotsubashi, Chiyoda-ku Tokyo 101-8430, Japan',
                        'Japan','6-10-1 Roppongi, Minato-ku Tokyo 106-6126','Inovallée, 655, Montbonnot 38334 Saint-Ismier Cedex','Tokyo']

p29pdf_references_authors =['A. A. Balkema','L. de Haan','N. Bingham', 'C. Goldie','J. Teugels','N. Boujemaa', 'J. Fauqueur', 'M. Ferecatu', 'F. Fleuret',
                            'V. Gouet', 'B. LeSaux','H. Sahbi','C. Bouveyron', 'G. Celeux', 'S. Girard','J. Bruske', 'G. Sommer',
                           'F. Camastra','A. Vinciarelli','S. Coles','J. Costa' ,'A. Hero','T. de Vries', 'S. Chawla','M. E. Houle',
                           'R. A. Fisher','L. H. C. Tippett','M. I. Fraga Alves', 'L. de Haan','T. Lin','M. I. Fraga Alves', 'M. I. Gomes','L. de Haan',
                           'B. V. Gnedenko',' A. Gupta', 'R. Krauthgamer','J. R. Lee','A. Gupta', 'R. Krauthgamer','J. R. Lee','M. Hein','J.-Y. Audibert',
                           'B. M. Hill','M. E. Houle','M. E. Houle','M. E. Houle','M. E. Houle', 'H. Kashima', 'M. Nett','M. E. Houle', 'X. Ma', 'M. Nett',
                            'V. Oria','M. E. Houle', 'X. Ma', 'V. Oria','J. Sun','M. E. Houle','M. Nett','H. Jegou', 'R. Tavenard', 'M. Douze','L. Amsaleg',
                           'I. Jollie','D. R. Karger','M. Ruhl','J. Karhunen','J. Joutsensalo','Y. LeCun', 'L. Bottou', 'Y. Bengio', 'P. Haner',
                           'J. Pickands, III','C. R. Rao','S. T. Roweis','L. K. Saul','A. Rozza', 'G. Lombardi', 'C. Ceruti', 'E. Casiraghi', 'P. Campadelli',
                           'B. Scholkopf', 'A. J. Smola','K.-R. Muller','U. Shaft','R. Ramakrishnan',' F. Takens','J. Tenenbaum', 'V. D. Silva','J. Langford',
                           'J. B. Tenenbaum', 'V. De Silva','J. C. Langford','J. B. Tenenbaum', 'V. De Silva','J. C. Langford','J. Venna','S. Kaski',
                           'P. Verveer','R. Duin','J. von Brunken', 'M. E. Houle', 'A. Zimek','J. von Brunken', 'M. E. Houle','A. Zimek']

p29pdf_allauthors=['Laurent Amsaleg','Stéphane Girard','Oussama Chelly','Teddy Furon','Michael E. Houle','Ken-ichi Kawarabayashi',
               'Michael Nett','A. A. Balkema','L. de Haan','N. Bingham', 'C. Goldie','J. Teugels','N. Boujemaa', 'J. Fauqueur', 'M. Ferecatu', 'F. Fleuret',
                            'V. Gouet', 'B. LeSaux','H. Sahbi','C. Bouveyron', 'G. Celeux', 'S. Girard','J. Bruske', 'G. Sommer',
                           'F. Camastra','A. Vinciarelli','S. Coles','J. Costa' ,'A. Hero','T. de Vries', 'S. Chawla','M. E. Houle',
                           'R. A. Fisher','L. H. C. Tippett','M. I. Fraga Alves', 'L. de Haan','T. Lin','M. I. Fraga Alves', 'M. I. Gomes','L. de Haan',
                           'B. V. Gnedenko',' A. Gupta', 'R. Krauthgamer','J. R. Lee','A. Gupta', 'R. Krauthgamer','J. R. Lee','M. Hein','J.-Y. Audibert',
                           'B. M. Hill','M. E. Houle','M. E. Houle','M. E. Houle','M. E. Houle', 'H. Kashima', 'M. Nett','M. E. Houle', 'X. Ma', 'M. Nett',
                            'V. Oria','M. E. Houle', 'X. Ma', 'V. Oria','J. Sun','M. E. Houle','M. Nett','H. Jegou', 'R. Tavenard', 'M. Douze','L. Amsaleg',
                           'I. Jollie','D. R. Karger','M. Ruhl','J. Karhunen','J. Joutsensalo','Y. LeCun', 'L. Bottou', 'Y. Bengio', 'P. Haner',
                           'J. Pickands, III','C. R. Rao','S. T. Roweis','L. K. Saul','A. Rozza', 'G. Lombardi', 'C. Ceruti', 'E. Casiraghi', 'P. Campadelli',
                           'B. Scholkopf', 'A. J. Smola','K.-R. Muller','U. Shaft','R. Ramakrishnan',' F. Takens','J. Tenenbaum', 'V. D. Silva','J. Langford',
                           'J. B. Tenenbaum', 'V. De Silva','J. C. Langford','J. B. Tenenbaum', 'V. De Silva','J. C. Langford','J. Venna','S. Kaski',
                           'P. Verveer','R. Duin','J. von Brunken', 'M. E. Houle', 'A. Zimek','J. von Brunken', 'M. E. Houle','A. Zimek']


print "There are %r authors" % len(p29pdf_authors)
print  # white space
print "There are %r author organizations" %len(p29pdf_author_organizations)
print 
print "There are %r author locations" % len(p29pdf_author_locations)
print  
print "There are %r authors in the references" %len(p29pdf_references_authors)

There are 7 authors

There are 6 author organizations

There are 8 author locations

There are 106 authors in the references


In [ ]:
re.sub('[\s]'," ",document)[9300:10500]
#regexp.search(re.sub('[\s]'," ",document)).group(1)[4900:6000]

In [ ]:
extraction(references)

In [ ]:
class entities(object):
  def __init__(self):
    self.persons = extraction(document)['persons']
    self.organizations = extraction(document)['organizations']

my_shape = entities()
my_shape.persons

# Code to extract emails

In [ ]:
import emailextractor
from emailextractor import file_to_str, get_emails
tuple(get_emails(document))

# Code to get only the Title and Author Names

In [146]:
p=re.compile('(.*)(?=ABSTRACT)')
p.search(re.sub('[\s]'," ",document)).group(1)

'Estimating Local Intrinsic Dimensionality  Laurent Amsaleg Equipe LINKMEDIA,  CNRS/IRISA Rennes, France  Campus Universitaire de  Beaulieu  35042 Rennes Cedex, France laurent.amsaleg@irisa.fr  Stphane Girard Equipe MISTIS, INRIA  Grenoble, France  Inovalle, 655, Montbonnot 38334 Saint-Ismier Cedex, stephane.girard@inria.fr  France  Teddy Furon  Equipe LINKMEDIA,  INRIA/IRISA Rennes, France  Campus Universitaire de  Beaulieu  35042 Rennes Cedex, France  teddy.furon@inria.fr  Ken-ichi Kawarabayashi  National Institute of Informatics, Japan 2-1-2 Hitotsubashi,  Chiyoda-ku  Tokyo 101-8430, Japan k_keniti@nii.ac.jp  Oussama Chelly National Institute of Informatics, Japan 2-1-2 Hitotsubashi,  Chiyoda-ku  Tokyo 101-8430, Japan  chelly@nii.ac.jp Michael E. Houle National Institute of Informatics, Japan 2-1-2 Hitotsubashi,  Chiyoda-ku  Tokyo 101-8430, Japan  meh@nii.ac.jp Michael Nett Google, Japan  6-10-1 Roppongi, Minato-ku  Tokyo 106-6126, Japan mnett@google.com  '

# Code to get Title Only (or most of it)

In [132]:
p=re.compile('(.+)(\\n\\n)')
q=re.compile('(?<=\\n\\n)(.+?)(?=\\n\\n)')
#p.search(document).group(1)+" "+q.search(document).group(1)
p.search(document).group(1)

'Estimating Local Intrinsic Dimensionality'

# Code to get Abstract only

In [ ]:
p= re.compile('(?<=ABSTRACT)(.+)(?=Categories and Subject Descriptors)')
p.search(re.sub('[\s]'," ",document)).group(1)

# Code to get keywords only

In [181]:
p= re.compile('(?<=Keywords)(.+)(?=INTRODUCTION)')
p.search(re.sub('[\s]'," ",document)).group(1)

' Frequent Itemsets; Pattern Mining; Rademacher Averages; Sampling; Statistical Learning Theory  1.  '

# Code to get Categories

In [178]:
p= re.compile('(?<=Categories and Subject Descriptors)(.+)(?=Keywords)')
re.findall('\[(.*?)\]',p.search(re.sub('[\s]'," ",document)).group(1))

['Database Management']

# Code to get Body only

In [108]:
p= re.compile('(?<=INTRODUCTION)(.+)(?=ACKNOWLEDGMENTS)')
p.search(re.sub('[\s]'," ",document)).group(1)

'  In an attempt to improve the discriminability of similar- ity measures, and the scalability of methods that depend on them, much attention has been given in the areas of machine learning, databases, and data mining to the development of dimensional reduction techniques. Linear techniques for di- mensionality reduction include Principal Component Anal- ysis (PCA) and its variants [4, 24]. Non-linear dimension- ality reduction methods  also known as manifold learn- ing techniques  include Isometric Mapping [36], Multi- Dimensional Scaling [35,37], Locally Linear Embedding and its variants [30], and Non-Linear Component Analysis [32]. Most reduction techniques require that a target dimension be provided by the user, although some attempt to deter- mine the dimension automatically. Ideally, the supplied di- mension should depend on the intrinsic dimensionality (ID) of the data. This has served to motivate the development of models of ID, as well as accurate estimators.  Over the past fe

# Code to get CONCLUSION only

In [183]:
p= re.compile('(?<=CONCLUSION)(.+)(?=ACKNOWLEDGMENTS)')
p.search(re.sub('[\s]'," ",document)).group(1)

'S  We present an algorithm for extracting a high-quality ap- proximation of the collection of FIs with probabilistic guar- antees. The algorithm employs progressive sampling with a stopping condition that relies on bounding the conditional Rademacher average of the problem using easy-to-compute characteristic quantities of the sample. The stopping con- dition can therefore be evaluated very eciently without the need to perform an expensive in-depth mining of the frequent itemsets in the sample at each step. To our knowl- edge this is the rst work that uses Rademacher averages in a knowledge discovery setting. The experimental results conrm that the algorithm is extremely successful at stop- ping fast at early iterations, and allows to extract very high- quality approximation of the collection of FIs. Among the possible directions for future work, it would be particularly interesting to better study the trade-o between the compu- tational complexity of the stopping condition and its ab

# Code to get References only

In [163]:
p= re.compile('(?<=REFERENCES)(.+)')
p.search(re.sub('[\s]'," ",document)).group(1)

'  [1] R. Agrawal and R. Srikant. Fast algorithms for mining  association rules in large databases. VLDB 94.  [2] R. Agrawal, T. Imieliski, and A. Swami. Mining asso- ciation rules between sets of items in large databases. SIGMOD Rec., 22:207216, June 1993.  [3] B. Gu, B. Liu, F. Hu, and H. Liu. Eciently determin- ing the starting sample size for progressive sampling. ECML01.  [4] P. L. Bartlett, S. Boucheron, and G. Lugosi. Model selection and error estimation. Mach. Learn., 48:85 113, 2002.  [5] S. Boucheron, O. Bousquet, and G. Lugosi. Theory of classication : A survey of some recent advances. ESAIM: Probability and Statistics, 9:323375, 2005.  [6] V. T. Chakaravarthy, V. Pandit, and Y. Sabharwal. Analysis of sampling techniques for association rule mining. ICDT09.  [7] B. Chen, P. Haas, and P. Scheuermann. A new two- phase sampling based algorithm for discovering associ- ation rules. KDD02.  [8] K.-T. Chuang, M.-S. Chen, and W.-C. Yang. Progres- sive sampling for association rules 

# Code to count the number of references

In [161]:
'''
create a match of all references by counting the number of integers enclosed by brackets (i.e. '[2]').  

This is how references are labeled in the research papers
'''

len(re.findall('\[(.*?)\]',regexp.search(re.sub('[\s]'," ",document)).group(1)))

30

In [177]:
re.findall('\[(.*?)\]',catsec)

['Database Management']